In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import shap
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio
import plotly.express as px

import catboost as cb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split, KFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, make_scorer, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectFromModel

import optuna

import warnings
warnings.filterwarnings('ignore')

In [2]:
class DataPreprocessing:
    def __init__(self, forest_path, no_forest_path):
        self.df_forest = pd.read_csv(forest_path)
        self.df_no_forest = pd.read_csv(no_forest_path)
        self.df = None

    def load_and_clean_data(self):
        # Drop 'Unnamed: 0' column
        self.df_no_forest.drop('Unnamed: 0', axis=1, inplace=True)
        self.df_forest.drop('Unnamed: 0', axis=1, inplace=True)

        # Concatenate dataframes
        self.df = pd.concat([self.df_forest, self.df_no_forest])

        # Drop specific IGBP categories
        IGBP_drop = ['BSV', 'WSA', 'SAV']
        self.df = self.df[~self.df['IGBP'].isin(IGBP_drop)]

        # Drop specific stations (Chosen by visual analysis)
        valid_stations = [
        'US_Ne1', 'BE_Vie', 'CA_Ca1', 'CA_Gro', 'CA_LP1', 'CA_NS1', 'CA_NS2', 
        'CA_NS3', 'CA_NS5', 'CA_Oas', 'CA_Obs', 'CA_Qfo', 'CA_SF2', 'CA_TP2', 
        'CA_TP3', 'CA_TPD', 'CH_Dav', 'CH_Lae', 'CN_Cha', 'CN_Qia', 'CZ_BK1', 
        'DE_Hai', 'DE_Lnf', 'DE_Tha', 'DK_Sor', 'FI_Hyy', 'FI_Let', 'FI_Var', 
        'FR_Fon', 'FR_LBr', 'IT_CA1', 'IT_CA3', 'IT_Col', 'IT_Lav', 'IT_Ren', 
        'IT_Ro1', 'IT_Ro2', 'IT_SRo', 'JP_SMF', 'RU_Fyo', 'SE_Kno', 'SE_Nor', 
        'UK_Ham', 'US_Blo', 'US_CMW', 'US_GLE', 'US_Ha1', 'US_Me3', 'US_MMS', 
        'US_NR1', 'US_Oho', 'US_Prr', 'US_Rpf', 'US_Syv', 'US_Uaf', 'US_UMB', 
        'US_UMd', 'US_WCr', 'AT_Neu', 'AU_DaP', 'AU_Fog', 'AU_Rig', 'AU_Stp', 
        'AU_Whr', 'AU_Wom', 'BR_Sa1', 'CA_NS6', 'CA_NS7', 'CH_Aws', 'CH_Fru', 
        'CH_Oe1', 'CH_Oe2', 'CL_SDF', 'CN_Dan', 'CN_Din', 'CN_Ha2', 'CN_HaM', 
        'CN_Sw2', 'CZ_BK2', 'CZ_wet', 'DE_Geb', 'DE_Gri', 'DE_Kli', 'DE_RuR', 
        'DE_SfN', 'DE_Spw', 'DE_Zrk', 'DK_Eng', 'ES_Amo', 'ES_LgS', 'FI_Jok', 
        'FI_Kaa', 'FI_Lom', 'FR_LGt', 'FR_Pue', 'GF_Guy', 'GL_NuF', 'GL_ZaH', 
        'IT_CA2', 'IT_Cp2', 'IT_Cpz', 'IT_MBo', 'IT_Noe', 'NL_Ca1', 'PA_SPs', 
        'PT_Esp', 'RU_Ha1', 'RU_Sam', 'RU_Tks', 'SE_Deg', 'SJ_Adv', 'US_ARb', 
        'US_ARc', 'US_ARM', 'US_Atq', 'US_EML', 'US_Goo', 'US_IB2', 'US_Ivo', 
        'US_KFS', 'US_KLS', 'US_Kon', 'US_Los', 'US_Myb', 'US_Ne2', 'US_Ne3', 
        'US_SRG', 'US_Sta', 'US_Tw1', 'US_Twt', 'US_Var', 'US_Whs', 'US_Wi6', 
        'US_Wi7', 'US_WPT']
    
        self.df = self.df[self.df['station'].isin(valid_stations)]

    def drop_rows_with_value_counts(self, threshold=5):
        """
        Drop rows in a Pandas DataFrame where values in a specific column
        appear more than the specified threshold times.
        """
        for station in self.df['station'].unique():
            station_data = self.df[self.df['station'] == station]
            value_counts = station_data['NEE'].value_counts()
            values_to_remove = value_counts[value_counts > threshold].index.tolist()
            self.df = self.df[~self.df['NEE'].isin(values_to_remove)]
        self.df.reset_index(drop=True, inplace=True)

    def plot_all_stations(self, df):
        for category in df['station'].unique():
            station_data = df[df['station'] == category]
            biom = station_data['IGBP'].unique()
            print(f'{category}')
            print(biom)
            plt.plot(station_data['NEE'])
            plt.xlabel('Date')
            plt.ylabel('Value')
            plt.title(f'Station: {category}')
            plt.show()
            plt.close()

    def correlation(self, dataset, threshold):
        col_corr = set()  # Set of all the names of correlated columns
        corr_matrix = dataset.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i]  # getting the name of column
                    col_corr.add(colname)
        return col_corr

    def plot_correlation_heatmap(self):
        plt.figure(figsize=(20, 10))
        sns.heatmap(self.df.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
        plt.title('Heatmap of Columns')
        plt.show()

    def replace_outliers_with_nan_rolling_iqr(self, window_size=3, scale=1.5):
        new_df = self.df.copy()
        for station in tqdm(self.df['station'].unique()):
            station_data = self.df[self.df['station'] == station]
            for column_name in station_data.columns:
                if column_name not in ['Year','Month','lon','lat','station','IGBP'] and np.issubdtype(self.df[column_name].dtype, np.number):
                    rolling_q1 = self.df[column_name].rolling(window=window_size, center=True).quantile(0.35)
                    rolling_q3 = self.df[column_name].rolling(window=window_size, center=True).quantile(0.65)
                    rolling_iqr = rolling_q3 - rolling_q1
                    
                    lower_bound = rolling_q1 - (scale * rolling_iqr)
                    upper_bound = rolling_q3 + (scale * rolling_iqr)
                    
                    outliers = (self.df[column_name] < lower_bound) | (self.df[column_name] > upper_bound)
                    
                    new_df.loc[outliers, column_name] = np.nan
            self.df = new_df

    def mean_daily_data_by_month(self):
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        df_date = self.df['Date']
        category = self.df['station']
        self.df.set_index('Date', inplace=True)

        cleaned_df = pd.DataFrame()
        for station in self.df['station'].unique():
            # Select data for the current station
            station_data = self.df[self.df['station'] == station]
            numerical_columns = station_data.select_dtypes(include=['number']).columns
            categorical_columns = station_data.select_dtypes(exclude=['number']).columns

            # Extract year and month from the index
            station_data['Year'] = station_data.index.year
            station_data['Month'] = station_data.index.month

            # Create the aggregation dictionary
            agg_dict = {col: 'mean' for col in numerical_columns}
            agg_dict.update({col: 'first' for col in categorical_columns})

            # Group by year and month, and calculate the mean for numerical columns
            result_df = station_data.groupby(['Year', 'Month']).agg(agg_dict).reset_index()
            cleaned_df = pd.concat([cleaned_df, result_df])
        return cleaned_df, df_date, category


In [3]:
data_preprocessing = DataPreprocessing(r'C:\Univercity\Skoltech\NEE_2.0\forest_meteo_gpp_nee_2024.csv', 
                                  r'C:\Univercity\Skoltech\NEE_2.0\NO_FOREST_meteo_gpp_nee_2024.csv')

data_preprocessing.load_and_clean_data()
data_preprocessing.drop_rows_with_value_counts()

corr_features = data_preprocessing.correlation(data_preprocessing.df.drop(['NEE', 'GPP'], axis=1), 0.6)
data_preprocessing.df.drop(corr_features, axis=1, inplace=True)

cleaned_df, df_date, category = data_preprocessing.mean_daily_data_by_month()
cleaned_df = cleaned_df.dropna(subset=['NEE'])
cleaned_df.reset_index(drop=True, inplace=True)
cleaned_df = cleaned_df.interpolate()

valid_features = ['GPP', 'IGBP', 'Z0M', 'RHOA', 'PW', 'ALLSKY_SRF_ALB',
                   'TS_RANGE', 'CLRSKY_SRF_ALB', 'WS50M_MIN', 'ZENITH_LUMINANCE']
columns_to_select = valid_features + ['NEE']

# Select the columns from the DataFrame
cleaned_df= cleaned_df[columns_to_select]

In [4]:
# Define the igbp_dict
igbp_dict = {
    0: "Water Bodies",
    1: "ENF",  # Evergreen Needleleaf Forests
    2: "EBF",  # Evergreen Broadleaf Forests
    3: "DNF",  # Deciduous Needleleaf Forests
    4: "DBF",  # Deciduous Broadleaf Forests
    5: "MF",   # Mixed Forests
    6: "CSH",  # Closed Shrublands
    7: "OSH",  # Open Shrublands
    8: "WSA",  # Woody Savannas
    9: "SAV",  # Savannas
    10: "GRA", # Grasslands
    11: "WET", # Permanent Wetlands
    12: "CRO", # Croplands
    13: "URB", # Urban and Built-up Lands
    14: "CNV", # Cropland/Natural Vegetation Mosaics
    15: "SNO", # Permanent Snow and Ice
    16: "BAR"  # Barren
}

# Invert the igbp_dict to map short names to integers
igbp_dict_inverted = {v: k for k, v in igbp_dict.items()}

# Replace the values in cleaned_df['IGBP']
cleaned_df['IGBP'] = cleaned_df['IGBP'].replace(igbp_dict_inverted)

In [5]:
# Features to drop
features_to_drop = ['NEE']
X = cleaned_df.drop(features_to_drop, axis=1)
y = cleaned_df['NEE']


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Specify the categorical features
categorical_features = ['IGBP']

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Separate categorical and numerical features
X_train_num = X_train.drop(columns=categorical_features)
X_test_num = X_test.drop(columns=categorical_features)

# Fit the scaler on the numerical training data and transform both training and testing numerical data
X_train_num_scaled = scaler.fit_transform(X_train_num)
X_test_num_scaled = scaler.transform(X_test_num)

# Convert scaled numerical data back to DataFrame
X_train_num_scaled_df = pd.DataFrame(X_train_num_scaled, columns=X_train_num.columns, index=X_train_num.index)
X_test_num_scaled_df = pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns, index=X_test_num.index)

# Concatenate scaled numerical data with categorical data
X_train_scaled = pd.concat([X_train_num_scaled_df, X_train[categorical_features]], axis=1)
X_test_scaled = pd.concat([X_test_num_scaled_df, X_test[categorical_features]], axis=1)

# Create Pool objects for training and testing
train_pool = Pool(data=X_train_scaled, label=y_train, cat_features=categorical_features)
test_pool = Pool(data=X_test_scaled, label=y_test, cat_features=categorical_features)


In [6]:
best_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
best_model.fit(train_pool)

# Make predictions
predictions = best_model.predict(test_pool)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

0:	learn: 1.9725284	total: 170ms	remaining: 2m 49s
100:	learn: 1.1104046	total: 2.85s	remaining: 25.3s
200:	learn: 0.9647791	total: 5.45s	remaining: 21.7s
300:	learn: 0.8814545	total: 8.22s	remaining: 19.1s
400:	learn: 0.8255284	total: 10.7s	remaining: 16s
500:	learn: 0.7846615	total: 13.4s	remaining: 13.3s
600:	learn: 0.7485362	total: 16.2s	remaining: 10.7s
700:	learn: 0.7177649	total: 19s	remaining: 8.12s
800:	learn: 0.6907762	total: 21.9s	remaining: 5.45s
900:	learn: 0.6650012	total: 24.8s	remaining: 2.72s
999:	learn: 0.6423793	total: 27.3s	remaining: 0us
Mean Squared Error: 1.0592574590782255
R-squared: 0.748411035716773


In [ ]:
# best_model.save_model('Catboost_NEE_model.cbm')